Loading the Dataset

In [104]:
import pandas as pd

df = pd.read_csv('GrammarandProductReviews.csv')
df.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


DATA SOURCING AND EXPLORATION

In [105]:
df.columns

Index(['id', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'keys',
       'manufacturer', 'manufacturerNumber', 'name', 'reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase',
       'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.userCity', 'reviews.userProvince', 'reviews.username', 'upc'],
      dtype='object')

In [106]:
# data sections

product_metadata = df[['id', 'brand', 'categories', 'manufacturer',
                       'manufacturerNumber', 'name', 'ean', 'upc', 'keys']]

review_metadata = df[['reviews.date',
       'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 
       'reviews.id', 'reviews.numHelpful',
       'reviews.sourceURLs']]

user_metadata = df[['reviews.userCity', 'reviews.userProvince', 'reviews.username']]

sentiment_data = df[['reviews.text', 'reviews.title','reviews.rating', 
                     'reviews.doRecommend']]

In [107]:
df.isnull().sum().sort_values(ascending=False)


reviews.userProvince    70595
reviews.userCity        65634
reviews.didPurchase     38886
reviews.id              38886
reviews.numHelpful      38536
ean                     31979
reviews.doRecommend     10615
reviews.title             477
manufacturerNumber        203
manufacturer              141
reviews.username          102
reviews.date               67
reviews.text               36
upc                         2
reviews.sourceURLs          0
reviews.rating              0
id                          0
brand                       0
reviews.dateAdded           0
name                        0
keys                        0
dateUpdated                 0
dateAdded                   0
categories                  0
reviews.dateSeen            0
dtype: int64

Text Cleaning and Preprocessing

In [108]:
# drop rows with missing reviews.text; can't do sentiment analysis without text
df = df.dropna(subset=['reviews.text'])

# fill missing values in brand and manufacturer with 'Unknown'
df['manufacturer'] = df['manufacturer'].fillna('Unknown')

'''
columns which doesnt affect analysis : 
ean, upc, manufacturerNumber, keys, reviews.sourceURLs (can be ignored)

columns which dont need to be imputed :
reviews.userCity, reviews.userProvince, reviews.username 

'''

df["reviews.title"]= df["reviews.title"].fillna('')
df["reviews.didPurchase"]= df["reviews.didPurchase"].fillna(False)
df["reviews.numHelpful"] = df["reviews.numHelpful"].fillna(0)

df.isnull().sum().sort_values(ascending=False)


C:\Users\Hemakshi Kumar\AppData\Local\Temp\ipykernel_27644\1276888914.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["reviews.didPurchase"]= df["reviews.didPurchase"].fillna(False)


reviews.userProvince    70559
reviews.userCity        65598
reviews.id              38852
ean                     31977
reviews.doRecommend     10581
manufacturerNumber        202
reviews.username          102
reviews.date               67
upc                         2
reviews.title               0
reviews.text                0
reviews.sourceURLs          0
reviews.rating              0
reviews.numHelpful          0
id                          0
reviews.didPurchase         0
brand                       0
reviews.dateAdded           0
name                        0
manufacturer                0
keys                        0
dateUpdated                 0
dateAdded                   0
categories                  0
reviews.dateSeen            0
dtype: int64

In [109]:

df['reviews.rating'].value_counts().sort_index(ascending=False)

reviews.rating
5    46516
4    14592
3     4367
2     1832
1     3701
Name: count, dtype: int64

In [110]:
def label_sentiment(rating):
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    else:
        return "negative"

df["sentiment"] = df["reviews.rating"].apply(label_sentiment)

In [111]:
# text preprocessing and lemmatization

df["full_review"] = df["reviews.title"] + " " + df["reviews.text"]

df.drop(columns=['reviews.title', 'reviews.text'], inplace=True)

df["full_review"] = (
    df["full_review"]
    .str.lower()
    .str.replace("http", "", regex=False)      # removes URL prefix
    .str.replace(r"[^a-z\s]", "", regex=True)  # keep only letters
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

import spacy
nlp = spacy.load(
    "en_core_web_sm",
    disable=["ner", "parser"]
)

df["full_review"] = [
    " ".join(
        token.lemma_
        for token in doc
        if token.is_alpha and not token.is_stop
    )
    for doc in nlp.pipe(df["full_review"], batch_size=2000)
]


df.head()

,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.userCity,reviews.userProvince,reviews.username,upc,sentiment,full_review
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,Los Angeles,NaN,Joshua,6.02537E+11,positive,awesome love album good hip hop current pop so...
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,0.0,5,https://www.walmart.com/reviews/product/29775278,NaN,NaN,Dorothy W,73416000391,positive,good good flavor review collect promotion
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,0.0,5,https://www.walmart.com/reviews/product/29775278,NaN,NaN,Dorothy W,73416000391,positive,good good flavor
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,0.0,1,https://www.walmart.com/reviews/product/43383370,NaN,NaN,Rebecca,67981934427,negative,disappointed read review look buy couple lubri...
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,0.0,1,https://www.walmart.com/reviews/product/43383370,NaN,NaN,Walker557,67981934427,negative,irritation husband buy gel gel cause irritatio...


Train-Test Split

In [112]:
from sklearn.model_selection import train_test_split
import numpy as np
x_text = df["full_review"]
y = df["sentiment"]
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
y = np.array([label_map[i] for i in y], dtype=np.int32)

x_train, x_test, y_train, y_test = train_test_split(
    x_text, y, test_size=0.2, random_state=42, stratify=y)

Tokenization & Embeddings

In [113]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>") # use top 20000 words only, replace others with <OOV> token
tokenizer.fit_on_texts(x_train) # fit tokenizer on training data

from tensorflow.keras.preprocessing.sequence import pad_sequences   

max_length = 200 # max length of sequences
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train_seq, maxlen=max_length, padding='post', truncating='post')
x_test_seq = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test_seq, maxlen=max_length, padding='post', truncating='post')


Model Design & Training

In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

vocab_size = min(20000, len(tokenizer.word_index) + 1)

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length, mask_zero=True),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dense(3, activation="softmax")
])

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam',
    metrics=['accuracy']
)


c:\Users\Hemakshi Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model Evaluation

In [115]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=1,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=64,
    callbacks = [early_stopping]
)


Epoch 1/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 159s 195ms/step - accuracy: 0.8787 - loss: 0.4051 - val_accuracy: 0.9032 - val_loss: 0.2758
Epoch 2/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 173s 217ms/step - accuracy: 0.9235 - loss: 0.2275 - val_accuracy: 0.9115 - val_loss: 0.2633
Epoch 3/5
799/799 ━━━━━━━━━━━━━━━━━━━━ 176s 221ms/step - accuracy: 0.9377 - loss: 0.1859 - val_accuracy: 0.9083 - val_loss: 0.2764


In [116]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)


444/444 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.9096 - loss: 0.2639
Test accuracy: 0.9109280109405518


In [117]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test).argmax(axis=1)
print(classification_report(y_test, y_pred))

444/444 ━━━━━━━━━━━━━━━━━━━━ 21s 46ms/step
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      1107
           1       0.53      0.17      0.25       873
           2       0.93      0.98      0.96     12222

    accuracy                           0.91     14202
   macro avg       0.73      0.62      0.65     14202
weighted avg       0.89      0.91      0.90     14202



Calculating Scores and Reordering Based on the Sentiment Analysis

In [118]:
# product-level sentiment analysis

df_test = df.iloc[-len(y_pred):].copy()

df_test["sentiment_pred"] = y_pred

df_test["sentiment_score"] = df_test["sentiment_pred"]

product_sentiment = (
    df_test.groupby(["name", "brand"], as_index=False)["sentiment_score"]
    .mean()
)

product_sentiment

,name,brand,sentiment_score
0,12110209 - Honeywell Quietset 8-Speed Tower Fa...,Honeywell,1.830508
1,16ct Peppa Pig Lunch Napkins,Peppa Pig,2.000000
2,"2x Ultra Era With Oxi Booster, 100fl Oz",Era,2.000000
3,5ct Lumabase Metallic Silver Round Paper Lante...,American Greetings,2.000000
4,Africa's Best No-Lye Dual Conditioning Relaxer...,Africa's Best,0.666667
...,...,...,...
126,"Wet N Wild Megalast Lip Color, Dollhouse Pink 967",Wet N Wild,2.000000
127,"Wheat Thins Crackers, Big",Wheat Thins,1.916667
128,Who Is Harry Nilsson (and Why Is Ever (dvd),Target.com Use Only,2.000000
129,Wish-Bone Greek Vinaigrette Salad Dressing,Wish-Bone,2.000000


In [119]:
base_recommendations = (
    df.groupby("name")
      .agg(
          avg_rating=("reviews.rating", "mean"),
          review_count=("reviews.rating", "count")
      )
      .reset_index()
)

base_recommendations


,name,avg_rating,review_count
0,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. ...,4.166667,6
1,100:Complete First Season (blu-Ray),4.712230,139
2,12110209 - Honeywell Quietset 8-Speed Tower Fa...,4.284746,295
3,16ct Peppa Pig Lunch Napkins,5.000000,2
4,2017-2018 Brownline174 Duraflex 14-Month Plann...,5.000000,4
...,...,...,...
593,Working Girl (dvd),4.593750,32
594,"Yardley Of London Bath and Shower Gel, Exfolia...",5.000000,1
595,Yes To Carrots Nourishing Body Wash,4.666667,66
596,Yes To Grapefruit Rejuvenating Body Wash,3.846154,39


In [120]:
final_recommendations = base_recommendations.merge(
    product_sentiment,
    on="name",
    how="left"
)

final_recommendations["sentiment_score"].fillna(0, inplace=True)

final_recommendations["final_score"] = (
    0.7 * final_recommendations["avg_rating"] +
    0.3 * final_recommendations["sentiment_score"]
)

final_recommendations.sort_values(by="final_score", ascending=True)


C:\Users\Hemakshi Kumar\AppData\Local\Temp\ipykernel_27644\244909873.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_recommendations["sentiment_score"].fillna(0, inplace=True)


,name,avg_rating,review_count,brand,sentiment_score,final_score
12,99179 - Fit System 01-05 Civic Replacement Mir...,1.0,1,NaN,0.0,0.7
565,Udi's Pepperoni Pizza,1.0,1,NaN,0.0,0.7
359,Newman's Own Beef & Broccoli Complete Skillet ...,1.0,1,NaN,0.0,0.7
424,Pocket Watch Wall Clock Distressed Black - Yos...,1.0,1,NaN,0.0,0.7
196,High-Dome Floor Door Stop,1.0,1,NaN,0.0,0.7
...,...,...,...,...,...,...
226,Jack Link's Teriyaki Beef Stick,5.0,3,Jack Link's,2.0,4.1
221,J. Hunt Bold Table Lamp,5.0,3,J. Hunt,2.0,4.1
213,Imagine Foods Organic Low Sodium Free Range Ch...,5.0,1,Imagine Foods,2.0,4.1
172,Ge Led 60watt Outdoor Post Light Bulb - Soft W...,5.0,2,General Electric,2.0,4.1


User Brand and Category Affinity

In [169]:
sentiment_map = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

df_test["sentiment_score"] = df_test["sentiment"].map(sentiment_map)

brand_affinity = (
    df_test.groupby(["reviews.username", "brand"])["sentiment_score"]
      .mean()
      .reset_index()
      .rename(columns={"sentiment_score": "brand_affinity"})
)

brand_affinity

,reviews.username,brand,brand_affinity
0,.45 Tom,Lemi Shine,2.0
1,05Shannon,FOX,0.0
2,10101010101,Sony Pictures,2.0
3,101proof,Sony Pictures,2.0
4,1031,Rubbermaid,2.0
...,...,...,...
13236,zorropirate,Rubbermaid,2.0
13237,zreene2100,Warner Bros.,2.0
13238,zulu9832,Paramount Pictures,2.0
13239,zunebug9211,Sony Pictures,2.0


In [170]:
df_test["primary_category"] = df_test["categories"].str.split(",").str[0]


category_affinity = (
    df_test
    .groupby(["reviews.username", "primary_category"])["sentiment_score"]
    .mean()
    .reset_index()
    .rename(columns={"sentiment_score": "category_affinity"})
)

category_affinity

,reviews.username,primary_category,category_affinity
0,.45 Tom,Household Essentials,2.0
1,05Shannon,Movies,0.0
2,10101010101,Movies,2.0
3,101proof,Movies,2.0
4,1031,Household Essentials,2.0
...,...,...,...
12663,zorropirate,Household Essentials,2.0
12664,zreene2100,Movies,2.0
12665,zulu9832,Movies,2.0
12666,zunebug9211,Movies,2.0


In [171]:
product_metadata = (df[["name", "brand", "categories"]].drop_duplicates())
product_metadata["primary_category"] = (product_metadata["categories"].str.split(",").str[0])
product_metadata.drop(columns=["categories"], inplace=True)
product_metadata.head()

,name,brand,primary_category
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),Universal Music,Movies
1,Lundberg Organic Cinnamon Toast Rice Cakes,Lundberg,Food
3,K-Y Love Sensuality Pleasure Gel,K-Y,Personal Care
30,"J.R. Watkins Hand Cream, Lemon Cream",J.R. Watkins,Personal Care
36,Ambi Complexion Cleansing Bar,AMBI,Personal Care


Sentiment-Aware Recommendation

In [172]:
def recommend_for_user(user, top_k=5):
    brand_aff = brand_affinity[brand_affinity["reviews.username"] == user]

    cat_aff = category_affinity[category_affinity["reviews.username"] == user]

    candidates = product_metadata.merge(brand_aff, on="brand", how="left") \
                             .merge(cat_aff, on="primary_category", how="left")

    candidates["brand_affinity"].fillna(0, inplace=True)
    candidates["category_affinity"].fillna(0, inplace=True)

    candidates["affinity_score"] = (
        0.6 * candidates["brand_affinity"] +
        0.4 * candidates["category_affinity"]
    )

    candidates = candidates.merge(
        final_recommendations[["name", "final_score"]],
        on="name",
        how="left"
    )

    candidates["sentiment_score"] = candidates["final_score"].fillna(0)
    candidates["final_score"] = (
        0.7 * candidates["sentiment_score"] +
        0.3 * candidates["affinity_score"]
    )

    candidates.drop(columns=["brand_affinity", "category_affinity", "reviews.username_x", "reviews.username_y"], inplace=True)
    print(candidates[candidates["affinity_score"] > 0].shape[0])
    return candidates.sort_values("final_score", ascending=False).head(top_k)


In [179]:
df["reviews.username"].value_counts().head(5)


reviews.username
An anonymous customer    421
Anonymous                113
Mike                     105
Chris                     88
ByAmazon Customer         58
Name: count, dtype: int64

Testing

In [178]:
recommend_for_user("An anonymous customer", 5)

333


C:\Users\Hemakshi Kumar\AppData\Local\Temp\ipykernel_27644\976578183.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  candidates["brand_affinity"].fillna(0, inplace=True)
C:\Users\Hemakshi Kumar\AppData\Local\Temp\ipykernel_27644\976578183.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

,name,brand,primary_category,affinity_score,final_score,sentiment_score
517,"2x Ultra Era With Oxi Booster, 100fl Oz",Era,Household Chemicals,2.000000,3.470000,4.100000
490,Maybelline Eye Studio Master Duo Glossy Liquid...,Maybelline,Personal Care,1.964444,3.459333,4.100000
483,"Suave Naturals Refreshing Body Wash, Ocean Breeze",Suave,Personal Care,1.964444,3.364495,3.964516
509,"Aveeno Stress Relief Moisturizing Lotion, 12oz",Aveeno,Personal Care,1.964444,3.314604,3.893243
514,Aveeno Positively Nourishing Purifying Body Wa...,Aveeno,Personal Care,1.964444,3.270872,3.830769
